# Simulation Analyser

## Import relevant libraries

In [1]:
import pandas as pd
import numpy as np

from configparser import ConfigParser

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import os
import quantstats as qs

import sqlite3
from sqlalchemy import create_engine

## Parameters

In [2]:
configur = ConfigParser()
config_file_name = "config_04a_mix_infer_s.ini"
print (configur.read(os.path.join('../config_files', config_file_name)))

['../config_files/config_04a_mix_infer_s.ini']


In [3]:
config_file_name_without_extension = config_file_name.replace(".ini", "")

ccy = configur.get('data', 'ccy')
data_frequency_train = configur.get('data', 'data_frequency_train')
data_frequency_val = configur.get('data', 'data_frequency_val')

window_size = configur.getint('environment', 'window_size')
start_date = configur.get('environment', 'start_date')
mid_date = configur.get('environment', 'mid_date')
end_date = configur.get('environment', 'end_date')
list_features = configur.get('environment', 'features').split(',')
target = configur.get('environment', 'target')

num_of_simulations = configur.getint('simulation', 'num_of_simulations')


## Get the data

In [4]:
def get_and_process_simulation_data(config_file_name_without_extension):
    try:
        
        # Get the data
        db_address = 'sqlite:///../results/data/db/simulation.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_simulation_results" + "_" + config_file_name_without_extension
        
        sql_command = "SELECT * FROM " + input_tbl_name

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        return df

    except Exception as e:
        print(e)

### Simulation Results

In [5]:
df_simulation_results = get_and_process_simulation_data(config_file_name_without_extension)

#### Plots

In [6]:
fig = px.box(df_simulation_results, y="fmt_total_profit_percentage", color="model_type", points="all", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600)
fig.show()

In [7]:
fig = px.histogram(df_simulation_results, x="fmt_total_profit_percentage", color="model_type", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600, marginal="rug", # can be `box`, `violin`
                         hover_data=['total_reward_cash','fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])
fig.show()

In [8]:
fig = px.histogram(df_simulation_results, x="num_of_trades", color="model_type", color_discrete_sequence=[ "#FF7F0E", "#00CC96", "#10aded", "#8A56EF"],  width=800, height=600, marginal="rug", # can be `box`, `violin`
                         hover_data=['total_reward_cash','fmt_total_profit_percentage', 'num_of_trades', 'sim_id'])
fig.show()

## Robot Actions & Env data for a particular simulation

#### Get the median of `fmt_total_profit_percentage` per model

In [9]:
df_sim_results_stats = df_simulation_results.groupby(['model_type'])['fmt_total_profit_percentage'].agg(['median']).reset_index()

#### Get the closest `sim_id` from the `fmt_total_profit_percentage` median

In [10]:
i_A2C = np.argmin(np.abs(df_simulation_results[df_simulation_results['model_type']=='A2C']['fmt_total_profit_percentage'] - df_sim_results_stats[df_sim_results_stats['model_type']=='A2C']['median'].values[0]))
median_sim_id_A2C = df_simulation_results[df_simulation_results['model_type']=='A2C'].iloc[i_A2C]['sim_id']

i_PPO = np.argmin(np.abs(df_simulation_results[df_simulation_results['model_type']=='PPO']['fmt_total_profit_percentage'] - df_sim_results_stats[df_sim_results_stats['model_type']=='PPO']['median'].values[0]))
median_sim_id_PPO = df_simulation_results[df_simulation_results['model_type']=='PPO'].iloc[i_PPO]['sim_id']

i_DQN = np.argmin(np.abs(df_simulation_results[df_simulation_results['model_type']=='DQN']['fmt_total_profit_percentage'] - df_sim_results_stats[df_sim_results_stats['model_type']=='DQN']['median'].values[0]))
median_sim_id_DQN = df_simulation_results[df_simulation_results['model_type']=='DQN'].iloc[i_DQN]['sim_id']

i_RANDOM = np.argmin(np.abs(df_simulation_results[df_simulation_results['model_type']=='RANDOM']['fmt_total_profit_percentage'] - df_sim_results_stats[df_sim_results_stats['model_type']=='RANDOM']['median'].values[0]))
median_sim_id_RANDOM = df_simulation_results[df_simulation_results['model_type']=='RANDOM'].iloc[i_RANDOM]['sim_id']

In [11]:
def get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id):
    try:
        
        # Get the data
        db_address = 'sqlite:///../results/data/db/simulation.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_robot_actions_and_env" + "_" + config_file_name_without_extension
        
        sql_command = "SELECT * FROM " + input_tbl_name
        
        sql_append = " WHERE " + "sim_id = " + str(sim_id)
        
        sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        return df

    except Exception as e:
        print(e)

In [12]:
def plot_robot_actions_all_models(config_file_name_without_extension, sim_id_A2C, sim_id_PPO, sim_id_DQN, sim_id_RANDOM):
    
    df_robot_actions_sim_id_A2C = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_A2C)
    df_robot_actions_sim_id_PPO = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_PPO)
    df_robot_actions_sim_id_DQN = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_DQN)
    df_robot_actions_sim_id_RANDOM = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_RANDOM)

    df_simulation_sim_id_A2C = df_simulation_results[df_simulation_results['sim_id'] == sim_id_A2C]
    df_simulation_sim_id_PPO = df_simulation_results[df_simulation_results['sim_id'] == sim_id_PPO]
    df_simulation_sim_id_DQN = df_simulation_results[df_simulation_results['sim_id'] == sim_id_DQN]
    df_simulation_sim_id_RANDOM = df_simulation_results[df_simulation_results['sim_id'] == sim_id_RANDOM]

    str_title_A2C = f"sim_id: {sim_id_A2C} | model_type: {df_simulation_sim_id_A2C.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_A2C.fmt_total_profit_percentage.values[0],2)}%"
    str_title_PPO = f"sim_id: {sim_id_PPO} | model_type: {df_simulation_sim_id_PPO.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_PPO.fmt_total_profit_percentage.values[0],2)}%"
    str_title_DQN = f"sim_id: {sim_id_DQN} | model_type: {df_simulation_sim_id_DQN.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_DQN.fmt_total_profit_percentage.values[0],2)}%"
    str_title_RANDOM = f"sim_id: {sim_id_RANDOM} | model_type: {df_simulation_sim_id_RANDOM.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_RANDOM.fmt_total_profit_percentage.values[0],2)}%"

    str_title = "The median profit percentage results per model"

    fig = make_subplots(rows=4, cols=1, subplot_titles=(str_title_A2C, str_title_PPO, str_title_DQN, str_title_RANDOM), shared_xaxes='all', shared_yaxes='all')

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_A2C["dates"], y=df_robot_actions_sim_id_A2C["prices"], mode='lines', marker_color='#969696', legendgroup = '1'), row=1, col=1)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '1'), row=1, col=1)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '1'), row=1, col=1)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_PPO["dates"], y=df_robot_actions_sim_id_PPO["prices"], mode='lines', marker_color='#969696', legendgroup = '2'), row=2, col=1)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '2'), row=2, col=1)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '2'), row=2, col=1)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_DQN["dates"], y=df_robot_actions_sim_id_DQN["prices"], mode='lines', marker_color='#969696', legendgroup = '3'), row=3, col=1)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '3'), row=3, col=1)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '3'), row=3, col=1)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_RANDOM["dates"], y=df_robot_actions_sim_id_RANDOM["prices"], mode='lines', marker_color='#969696', legendgroup = '4'), row=4, col=1)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '4'), row=4, col=1)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '4'), row=4, col=1)

    fig.update_xaxes(title_text = "Dates", title_standoff = 25)

    fig.update_yaxes(title_text = "", title_standoff = 25)

    fig.update_layout(title=str_title, title_x=0.5, width=1200, height=800, legend_tracegroupgap=125)
    fig.show()

In [13]:
plot_robot_actions_all_models(config_file_name_without_extension, median_sim_id_A2C, median_sim_id_PPO, median_sim_id_DQN, median_sim_id_RANDOM)

### QuantStat Analysis

In [14]:
# qs.extend_pandas()

# df_temp = df_robot_actions_and_env_one_sim_id.copy()
# df_temp['Date'] = pd.to_datetime(df_temp['dates'])
# df_temp.set_index('Date', inplace=True)

# net_worth = pd.Series(df_temp['total_profit_percentage'], index=df_temp.index[6:])
# returns = net_worth.pct_change().iloc[1:]

# qs.reports.full(returns)

## ALL: Robot Actions & Env data for a particular simulation

### Get the data

In [14]:
def get_and_process_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension):
    try:
        
        # Get the data
        db_address = 'sqlite:///../results/data/db/simulation.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_robot_actions_and_env" + "_" + config_file_name_without_extension
        
        sql_command = "SELECT * FROM " + input_tbl_name + " AS t1 "
        
        sql_append = "LEFT JOIN (SELECT sim_id, model_type FROM tbl_simulation_results_config_03a_mix_infer) AS t2 ON t1.sim_id = t2.sim_id"
        
        sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        return df

    except Exception as e:
        print(e)

In [15]:
def plot_robot_actions_all_sim_id_all_models(config_file_name_without_extension, sim_id_A2C, sim_id_PPO, sim_id_DQN, sim_id_RANDOM):
    
    df_robot_actions_and_env_all_sim_id = get_and_process_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension)

    df_robot_actions_and_env_all_sim_id_A2C = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'A2C']
    df_robot_actions_and_env_all_sim_id_PPO = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'PPO']
    df_robot_actions_and_env_all_sim_id_DQN = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'DQN']
    df_robot_actions_and_env_all_sim_id_RANDOM = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'RANDOM']    

    df_model_type_action_count_A2C = df_robot_actions_and_env_all_sim_id_A2C.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_A2C.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    df_model_type_action_count_PPO = df_robot_actions_and_env_all_sim_id_PPO.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_PPO.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    df_model_type_action_count_DQN = df_robot_actions_and_env_all_sim_id_DQN.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_DQN.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    df_model_type_action_count_RANDOM = df_robot_actions_and_env_all_sim_id_RANDOM.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_RANDOM.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)        

    df_robot_actions_sim_id_A2C = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_A2C)
    df_robot_actions_sim_id_PPO = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_PPO)
    df_robot_actions_sim_id_DQN = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_DQN)
    df_robot_actions_sim_id_RANDOM = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_RANDOM)

    df_simulation_sim_id_A2C = df_simulation_results[df_simulation_results['sim_id'] == sim_id_A2C]
    df_simulation_sim_id_PPO = df_simulation_results[df_simulation_results['sim_id'] == sim_id_PPO]
    df_simulation_sim_id_DQN = df_simulation_results[df_simulation_results['sim_id'] == sim_id_DQN]
    df_simulation_sim_id_RANDOM = df_simulation_results[df_simulation_results['sim_id'] == sim_id_RANDOM]

    str_title_A2C = f"Median sim_id: {sim_id_A2C} | model_type: {df_simulation_sim_id_A2C.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_A2C.fmt_total_profit_percentage.values[0],2)}%"
    str_title_PPO = f"Median sim_id: {sim_id_PPO} | model_type: {df_simulation_sim_id_PPO.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_PPO.fmt_total_profit_percentage.values[0],2)}%"
    str_title_DQN = f"Median sim_id: {sim_id_DQN} | model_type: {df_simulation_sim_id_DQN.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_DQN.fmt_total_profit_percentage.values[0],2)}%"
    str_title_RANDOM = f"Median sim_id: {sim_id_RANDOM} | model_type: {df_simulation_sim_id_RANDOM.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_RANDOM.fmt_total_profit_percentage.values[0],2)}%"

    str_title = f"Median Total Profit Results x Overall view of actions (across all the simulations)"

    fig = make_subplots(rows=4, cols=1, subplot_titles=(str_title_A2C, str_title_PPO, str_title_DQN, str_title_RANDOM), shared_xaxes='all', shared_yaxes='all', specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]]) 

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_A2C["dates"], y=df_robot_actions_sim_id_A2C["prices"], mode='lines', marker_color='#969696', legendgroup = '1'), row=1, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '1'), row=1, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '1'), row=1, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '1'), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '1'), row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_PPO["dates"], y=df_robot_actions_sim_id_PPO["prices"], mode='lines', marker_color='#969696', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '2'), row=2, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '2'), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '2'), row=2, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_DQN["dates"], y=df_robot_actions_sim_id_DQN["prices"], mode='lines', marker_color='#969696', legendgroup = '3'), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '3'), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '3'), row=3, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '3'), row=3, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '3'), row=3, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_RANDOM["dates"], y=df_robot_actions_sim_id_RANDOM["prices"], mode='lines', marker_color='#969696', legendgroup = '4'), row=4, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '4'), row=4, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '4'), row=4, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '4'), row=4, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '4'), row=4, col=1, secondary_y=False)

    fig.update_yaxes(title_text="Proportion of trades<br>across all the simulations", secondary_y=False)
    fig.update_yaxes(title_text=f"CCY level with the<br>sim_id actions related", secondary_y=True)

    fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'}, title=str_title, title_x=0.5, width=1200, height=800, legend_tracegroupgap=82, font={'size': 8})

    fig.show()

    

In [16]:
plot_robot_actions_all_sim_id_all_models(config_file_name_without_extension, median_sim_id_A2C, median_sim_id_PPO, median_sim_id_DQN, median_sim_id_RANDOM)

## Positive range: Robot Actions & Env data for a particular simulation

### Get the data

In [20]:
def get_and_process_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension):
    try:
        
        # Get the data
        db_address = 'sqlite:///../results/data/db/simulation.db'
        engine = create_engine(db_address, echo=False)
        sqlite_connection = engine.connect()

        input_tbl_name = "tbl_robot_actions_and_env" + "_" + config_file_name_without_extension
        
        sql_command = "SELECT dates, prices, actions, total_reward_cash, total_profit_percentage, fmt_total_profit_percentage, num_of_trades, t1.sim_id, t2.model_type FROM " + input_tbl_name + " AS t1 "
        
        sql_append = "LEFT JOIN (SELECT sim_id, model_type FROM tbl_simulation_results_" + config_file_name_without_extension + ") AS t2 ON t1.sim_id = t2.sim_id"
        
        sql_command = sql_command + sql_append

        df = pd.read_sql(sql_command, sqlite_connection)

        sqlite_connection.close()

        return df

    except Exception as e:
        print(e)

In [21]:
def plot_robot_actions_all_sim_id_all_models_positive_profit(config_file_name_without_extension, sim_id_A2C, sim_id_PPO, sim_id_DQN, sim_id_RANDOM):
    
    df_robot_actions_and_env_all_sim_id = get_and_process_robot_actions_and_env_data_all_sim_id(config_file_name_without_extension)

    df_robot_actions_and_env_all_sim_id_A2C = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'A2C']
    df_robot_actions_and_env_all_sim_id_PPO = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'PPO']
    df_robot_actions_and_env_all_sim_id_DQN = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'DQN']
    df_robot_actions_and_env_all_sim_id_RANDOM = df_robot_actions_and_env_all_sim_id[df_robot_actions_and_env_all_sim_id['model_type'] == 'RANDOM']    

    df_simulation_A2C = df_simulation_results[df_simulation_results['model_type'] == 'A2C']
    df_simulation_PPO = df_simulation_results[df_simulation_results['model_type'] == 'PPO']
    df_simulation_DQN = df_simulation_results[df_simulation_results['model_type'] == 'DQN']
    df_simulation_RANDOM = df_simulation_results[df_simulation_results['model_type'] == 'RANDOM']

    df_simulation_A2C_filtered = df_simulation_A2C[(df_simulation_A2C.fmt_total_profit_percentage > 0)]
    df_simulation_PPO_filtered = df_simulation_PPO[(df_simulation_PPO.fmt_total_profit_percentage > 0)]
    df_simulation_DQN_filtered = df_simulation_DQN[(df_simulation_DQN.fmt_total_profit_percentage > 0)]
    df_simulation_RANDOM_filtered = df_simulation_RANDOM[(df_simulation_RANDOM.fmt_total_profit_percentage > 0)]

    df_robot_actions_and_env_all_sim_id_A2C_filtered = df_robot_actions_and_env_all_sim_id_A2C[df_robot_actions_and_env_all_sim_id_A2C.sim_id.isin(df_simulation_A2C_filtered.sim_id)]
    df_robot_actions_and_env_all_sim_id_PPO_filtered = df_robot_actions_and_env_all_sim_id_PPO[df_robot_actions_and_env_all_sim_id_PPO.sim_id.isin(df_simulation_PPO_filtered.sim_id)]
    df_robot_actions_and_env_all_sim_id_DQN_filtered = df_robot_actions_and_env_all_sim_id_DQN[df_robot_actions_and_env_all_sim_id_DQN.sim_id.isin(df_simulation_DQN_filtered.sim_id)]
    df_robot_actions_and_env_all_sim_id_RANDOM_filtered = df_robot_actions_and_env_all_sim_id_RANDOM[df_robot_actions_and_env_all_sim_id_RANDOM.sim_id.isin(df_simulation_RANDOM_filtered.sim_id)]

    df_model_type_action_count_A2C = df_robot_actions_and_env_all_sim_id_A2C_filtered.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_A2C.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    df_model_type_action_count_PPO = df_robot_actions_and_env_all_sim_id_PPO_filtered.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_PPO.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    df_model_type_action_count_DQN = df_robot_actions_and_env_all_sim_id_DQN_filtered.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_DQN.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)

    df_model_type_action_count_RANDOM = df_robot_actions_and_env_all_sim_id_RANDOM_filtered.assign(val=1).pivot_table(values='val', index=['dates'], columns=['model_type', 'actions'], aggfunc='count').stack(0).reset_index()
    df_model_type_action_count_RANDOM.rename(columns = {0.0:'action_short_count', 1.0:'action_long_count'}, inplace = True)        

    df_robot_actions_sim_id_A2C = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_A2C)
    df_robot_actions_sim_id_PPO = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_PPO)
    df_robot_actions_sim_id_DQN = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_DQN)
    df_robot_actions_sim_id_RANDOM = get_and_process_robot_actions_and_env_data_one_sim_id(config_file_name_without_extension, sim_id_RANDOM)

    df_simulation_sim_id_A2C = df_simulation_results[df_simulation_results['sim_id'] == sim_id_A2C]
    df_simulation_sim_id_PPO = df_simulation_results[df_simulation_results['sim_id'] == sim_id_PPO]
    df_simulation_sim_id_DQN = df_simulation_results[df_simulation_results['sim_id'] == sim_id_DQN]
    df_simulation_sim_id_RANDOM = df_simulation_results[df_simulation_results['sim_id'] == sim_id_RANDOM]

    str_title_A2C = f"Median sim_id: {sim_id_A2C} | model_type: {df_simulation_sim_id_A2C.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_A2C.fmt_total_profit_percentage.values[0],2)}%"
    str_title_PPO = f"Median sim_id: {sim_id_PPO} | model_type: {df_simulation_sim_id_PPO.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_PPO.fmt_total_profit_percentage.values[0],2)}%"
    str_title_DQN = f"Median sim_id: {sim_id_DQN} | model_type: {df_simulation_sim_id_DQN.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_DQN.fmt_total_profit_percentage.values[0],2)}%"
    str_title_RANDOM = f"Median sim_id: {sim_id_RANDOM} | model_type: {df_simulation_sim_id_RANDOM.model_type.values[0]} | Total Profit percentage {round(df_simulation_sim_id_RANDOM.fmt_total_profit_percentage.values[0],2)}%"

    str_title = f"Median Total Profit Results x Overall view of actions (across all simulations with positive results)"

    fig = make_subplots(rows=4, cols=1, subplot_titles=(str_title_A2C, str_title_PPO, str_title_DQN, str_title_RANDOM), shared_xaxes='all', specs=[[{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}], [{"secondary_y": True}]]) 

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_A2C["dates"], y=df_robot_actions_sim_id_A2C["prices"], mode='lines', marker_color='#969696', legendgroup = '1'), row=1, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '1'), row=1, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['dates'], y=df_robot_actions_sim_id_A2C[df_robot_actions_sim_id_A2C['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '1'), row=1, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '1'), row=1, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_A2C['dates'], y=df_model_type_action_count_A2C['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '1'), row=1, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_PPO["dates"], y=df_robot_actions_sim_id_PPO["prices"], mode='lines', marker_color='#969696', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '2'), row=2, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['dates'], y=df_robot_actions_sim_id_PPO[df_robot_actions_sim_id_PPO['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '2'), row=2, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '2'), row=2, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_PPO['dates'], y=df_model_type_action_count_PPO['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '2'), row=2, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_DQN["dates"], y=df_robot_actions_sim_id_DQN["prices"], mode='lines', marker_color='#969696', legendgroup = '3'), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '3'), row=3, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['dates'], y=df_robot_actions_sim_id_DQN[df_robot_actions_sim_id_DQN['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '3'), row=3, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '3'), row=3, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_DQN['dates'], y=df_model_type_action_count_DQN['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '3'), row=3, col=1, secondary_y=False)

    fig.add_trace(go.Scatter(name='Price', x=df_robot_actions_sim_id_RANDOM["dates"], y=df_robot_actions_sim_id_RANDOM["prices"], mode='lines', marker_color='#969696', legendgroup = '4'), row=4, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Long', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 1]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#228b22', legendgroup = '4'), row=4, col=1, secondary_y=True)
    fig.add_trace(go.Scatter(name='Short', x=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['dates'], y=df_robot_actions_sim_id_RANDOM[df_robot_actions_sim_id_RANDOM['actions'] == 0]['prices'], mode='markers', marker_symbol='circle', marker_size=10, marker_color='#c83232', legendgroup = '4'), row=4, col=1, secondary_y=True)

    fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_long_count'], name='Long', marker_color=' #b3ffb3', legendgroup = '4'), row=4, col=1, secondary_y=False)
    fig.add_trace(go.Bar(x=df_model_type_action_count_RANDOM['dates'], y=df_model_type_action_count_RANDOM['action_short_count'], name='Short', marker_color='#ffb2b2', legendgroup = '4'), row=4, col=1, secondary_y=False)

    fig.update_yaxes(title_text="Proportion of trades<br>across all the simulations", secondary_y=False)
    fig.update_yaxes(title_text=f"CCY level with the<br>sim_id actions related", secondary_y=True)

    fig.update_layout(barmode='stack', xaxis={'categoryorder':'category ascending'}, title=str_title, title_x=0.5, width=1200, height=800, legend_tracegroupgap=82, font={'size': 8})

    fig.show()

    

In [22]:
plot_robot_actions_all_sim_id_all_models_positive_profit(config_file_name_without_extension, median_sim_id_A2C, median_sim_id_PPO, median_sim_id_DQN, median_sim_id_RANDOM)

KeyError: 'action_long_count'